## DIA 108: Enviar Automáticamente el Gráfico del Dashboard por Correo Semanalmente

Hoy aprenderás a capturar el gráfico del dashboard generado por Chart.js, convertirlo en una imagen, insertarlo en un PDF y enviarlo automáticamente por correo, ¡cada semana!

✅ ¿Qué aprenderás hoy?

🎯 Renderizar un gráfico en servidor.

🖼️ Insertarlo en un PDF con reportlab.

📬 Enviarlo como archivo adjunto por email usando smtplib.

🕒 Automatizar con schedule.

🧩 1. Requisitos
Instala estas librerías si no las tienes:

bash
Copiar
Editar
pip install matplotlib reportlab schedule
🧩 2. Crear el gráfico en servidor (usando matplotlib)
python
Copiar
Editar
import matplotlib.pyplot as plt
import io

def generar_grafico_predicciones(registros):
    fechas = [r.fecha.strftime('%d-%m %H:%M') for r in registros]
    predicciones = [int(r.prediccion) for r in registros]

    plt.figure(figsize=(10, 4))
    plt.plot(fechas, predicciones, marker='o', color='blue')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.title("📈 Predicciones Semanales")
    plt.xlabel("Fecha")
    plt.ylabel("Predicción")

    buffer = io.BytesIO()
    plt.savefig(buffer, format='PNG')
    buffer.seek(0)
    plt.close()

    return buffer
🧩 3. Insertar gráfico en PDF
python
Copiar
Editar
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

def generar_pdf_con_grafico(user, registros):
    imagen_buffer = generar_grafico_predicciones(registros)
    pdf_buffer = io.BytesIO()
    p = canvas.Canvas(pdf_buffer, pagesize=letter)
    width, height = letter

    p.setFont("Helvetica-Bold", 14)
    p.drawString(50, height - 50, f"Reporte de Predicciones - {user.username}")
    p.drawImage(ImageReader(imagen_buffer), 50, height - 350, width=500, height=250)
    p.showPage()
    p.save()
    pdf_buffer.seek(0)
    return pdf_buffer
🧩 4. Enviar PDF por correo
python
Copiar
Editar
import smtplib
from email.message import EmailMessage
from dotenv import load_dotenv
import os

load_dotenv()

def enviar_email_con_pdf(user, pdf_buffer):
    msg = EmailMessage()
    msg['Subject'] = f'Reporte Semanal - {user.username}'
    msg['From'] = os.getenv("EMAIL_SENDER")
    msg['To'] = user.email

    msg.set_content(f"""
Hola {user.username},

Adjunto encontrarás tu reporte semanal de predicciones.

Saludos,
Tu Sistema de Predicción
""")

    msg.add_attachment(pdf_buffer.read(), maintype='application', subtype='pdf',
                       filename=f'reporte_{user.username}.pdf')

    try:
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(os.getenv("EMAIL_SENDER"), os.getenv("EMAIL_PASSWORD"))
            smtp.send_message(msg)
        print(f"✅ Enviado a {user.email}")
    except Exception as e:
        print(f"❌ Error al enviar a {user.email}: {e}")
🧩 5. Programar el envío automático semanal
python
Copiar
Editar
import schedule
import time
from app import db, app, User, Registro
from datetime import datetime, timedelta

def tarea_semanal():
    with app.app_context():
        hoy = datetime.today()
        hace_7_dias = hoy - timedelta(days=7)

        usuarios = User.query.all()
        for user in usuarios:
            registros = Registro.query.filter(
                Registro.user_id == user.id,
                Registro.fecha >= hace_7_dias
            ).order_by(Registro.fecha).all()

            if registros:
                pdf = generar_pdf_con_grafico(user, registros)
                enviar_email_con_pdf(user, pdf)

schedule.every().monday.at("08:00").do(tarea_semanal)

print("⏳ Esperando envío semanal de gráficos...")

while True:
    schedule.run_pending()
    time.sleep(60)